In [50]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
#from sklearn.cross_validation import KFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
lemma=WordNetLemmatizer()
stop=set(stopwords.words('english'))
import seaborn as sns
import pandas as pd

In [51]:
cd_train=pd.read_csv('Consumer_Complaints_train.csv')
cd_test=pd.read_csv('Consumer_Complaints_test_share.csv')

In [52]:
cd_train.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572


In [53]:
cd_train['Consumer disputed?']=np.where(cd_train['Consumer disputed?']=="Yes",0,1)

In [54]:
cd_train.isna().mean().round(4) * 100

Date received                    0.00
Product                          0.00
Sub-product                     28.94
Issue                            0.00
Sub-issue                       61.16
Consumer complaint narrative    84.30
Company public response         81.11
Company                          0.00
State                            0.80
ZIP code                         0.80
Tags                            85.95
Consumer consent provided?      71.68
Submitted via                    0.00
Date sent to company             0.00
Company response to consumer     0.00
Timely response?                 0.00
Consumer disputed?               0.00
Complaint ID                     0.00
dtype: float64

In [55]:
cd_train['Product'].value_counts()

Mortgage                   156175
Debt collection             86544
Credit reporting            81115
Credit card                 57358
Bank account or service     54403
Consumer Loan               18599
Student loan                14918
Money transfers              3349
Payday loan                  3219
Prepaid card                 2226
Other financial service       507
Virtual currency                8
Name: Product, dtype: int64

In [56]:
for col in ['Date received','Date sent to company']:
    cd_train[col]=pd.to_datetime(cd_train[col],infer_datetime_format=True)
    cd_test[col]=pd.to_datetime(cd_test[col],infer_datetime_format=True)

In [57]:
cd_train['day_diff']=pd.to_numeric((cd_train['Date sent to company']-cd_train['Date received'])/np.timedelta64(1, 'M'))
cd_test['day_diff']=pd.to_numeric((cd_test['Date sent to company']-cd_test['Date received'])/np.timedelta64(1, 'M'))

In [58]:
for col in ['Date received','Date sent to company']:
    cd_train.drop([col],1,inplace=True)
    cd_test.drop([col],1,inplace=True)

In [59]:
for col in cd_train.select_dtypes(['object']).columns:
    print(col,':',cd_train[col].nunique())

Product : 12
Sub-product : 47
Issue : 95
Sub-issue : 68
Consumer complaint narrative : 74019
Company public response : 10
Company : 3276
State : 62
ZIP code : 25962
Tags : 3
Consumer consent provided? : 4
Submitted via : 6
Company response to consumer : 7
Timely response? : 2


In [60]:
for col in cd_test.select_dtypes(['object']).columns:
    print(col,':',cd_test[col].nunique())

Product : 12
Sub-product : 47
Issue : 94
Sub-issue : 67
Consumer complaint narrative : 18438
Company public response : 10
Company : 2237
State : 62
ZIP code : 17784
Tags : 3
Consumer consent provided? : 4
Submitted via : 6
Company response to consumer : 7
Timely response? : 2


In [61]:
cd_train['Consumer disputed?'].value_counts()

1    376990
0    101431
Name: Consumer disputed?, dtype: int64

In [62]:
cd_train.isnull().sum()

Product                              0
Sub-product                     138473
Issue                                0
Sub-issue                       292625
Consumer complaint narrative    403327
Company public response         388029
Company                              0
State                             3839
ZIP code                          3848
Tags                            411215
Consumer consent provided?      342934
Submitted via                        0
Company response to consumer         0
Timely response?                     0
Consumer disputed?                   0
Complaint ID                         0
day_diff                             0
dtype: int64

In [63]:
for col in ['Tags','Company public response']:
    varname=col.replace('-','_').replace('?','').replace(" ",'_')+'_isNan'
    cd_train[varname]=np.where(pd.isnull(cd_train[col]),1,0)
    cd_train.drop([col],1,inplace=True)
    cd_test[varname]=np.where(pd.isnull(cd_test[col]),1,0)
    cd_test.drop([col],1,inplace=True)
    

In [64]:
cd_train.dtypes

Product                           object
Sub-product                       object
Issue                             object
Sub-issue                         object
Consumer complaint narrative      object
Company                           object
State                             object
ZIP code                          object
Consumer consent provided?        object
Submitted via                     object
Company response to consumer      object
Timely response?                  object
Consumer disputed?                 int32
Complaint ID                       int64
day_diff                         float64
Tags_isNan                         int32
Company_public_response_isNan      int32
dtype: object

In [65]:
for col in cd_train.select_dtypes(['object']).columns:
    print(col,':',cd_train[col].nunique())

Product : 12
Sub-product : 47
Issue : 95
Sub-issue : 68
Consumer complaint narrative : 74019
Company : 3276
State : 62
ZIP code : 25962
Consumer consent provided? : 4
Submitted via : 6
Company response to consumer : 7
Timely response? : 2


In [66]:
k=cd_train['State'].value_counts()
for val in k.axes[0][0:15]:
    varname='State_'+val.replace(',','_').replace(' ','_')
    cd_train[varname]=np.where(cd_train['State']==val,1,0)
    cd_test[varname]=np.where(cd_test['State']==val,1,0)
del cd_train['State']
del cd_test['State']

In [67]:
for col in ['Product','Submitted via','Company response to consumer','Timely response?']:
    
    temp=pd.get_dummies(cd_train[col],prefix=col,drop_first=True)
    cd_train=pd.concat([temp,cd_train],1)
    cd_train.drop([col],1,inplace=True)
    
    temp=pd.get_dummies(cd_test[col],prefix=col,drop_first=True)
    cd_test=pd.concat([temp,cd_test],1)
    cd_test.drop([col],1,inplace=True)


In [68]:
for col in cd_train.select_dtypes(['object']).columns:
    print(col,':',cd_train[col].nunique())

Sub-product : 47
Issue : 95
Sub-issue : 68
Consumer complaint narrative : 74019
Company : 3276
ZIP code : 25962
Consumer consent provided? : 4


In [69]:
cd_train.shape

(478421, 50)

In [70]:
cd_test.shape

(119606, 49)

In [71]:
for col in ['ZIP code','Company']:
    cd_train.drop([col],1,inplace=True)
    cd_test.drop([col],1,inplace=True)

In [72]:
for col in cd_train.select_dtypes(['object']).columns:
    print(col,':',cd_train[col].nunique())

Sub-product : 47
Issue : 95
Sub-issue : 68
Consumer complaint narrative : 74019
Consumer consent provided? : 4


In [73]:
for col in cd_test.select_dtypes(['object']).columns:
    print(col,':',cd_test[col].nunique())

Sub-product : 47
Issue : 94
Sub-issue : 67
Consumer complaint narrative : 18438
Consumer consent provided? : 4


In [74]:
cd_train['Super']=cd_train['Consumer consent provided?']+cd_train['Sub-product']+cd_train['Issue']+','+cd_train['Sub-issue']+','+cd_train['Consumer complaint narrative']
del cd_train['Issue']
del cd_train['Sub-issue']
del cd_train['Consumer complaint narrative']
del cd_train['Consumer consent provided?']
cd_train["Super"]=np.where(pd.isnull(cd_train["Super"]),'NaN',cd_train['Super'])
cd_train['Super'].astype(str)
    

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
478416    NaN
478417    NaN
478418    NaN
478419    NaN
478420    NaN
Name: Super, Length: 478421, dtype: object

In [75]:
cd_test['Super']=cd_test['Consumer consent provided?']+cd_test['Sub-product']+cd_test['Issue']+','+cd_test['Sub-issue']+','+cd_test['Consumer complaint narrative']
del cd_test['Issue']
del cd_test['Sub-issue']
del cd_test['Consumer consent provided?']
del cd_test['Consumer complaint narrative']
cd_test["Super"]=np.where(pd.isnull(cd_test["Super"]),'NaN',cd_test['Super'])
cd_test['Super'].astype(str)
    

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
119601    NaN
119602    NaN
119603    NaN
119604    NaN
119605    NaN
Name: Super, Length: 119606, dtype: object

In [76]:
cd_test.isnull().sum()

Timely response?_Yes                                                0
Company response to consumer_Closed with explanation                0
Company response to consumer_Closed with monetary relief            0
Company response to consumer_Closed with non-monetary relief        0
Company response to consumer_Closed with relief                     0
Company response to consumer_Closed without relief                  0
Company response to consumer_Untimely response                      0
Submitted via_Fax                                                   0
Submitted via_Phone                                                 0
Submitted via_Postal mail                                           0
Submitted via_Referral                                              0
Submitted via_Web                                                   0
Product_Consumer Loan                                               0
Product_Credit card                                                 0
Product_Credit repor

In [77]:
cd_train.isnull().sum()

Timely response?_Yes                                                 0
Company response to consumer_Closed with explanation                 0
Company response to consumer_Closed with monetary relief             0
Company response to consumer_Closed with non-monetary relief         0
Company response to consumer_Closed with relief                      0
Company response to consumer_Closed without relief                   0
Company response to consumer_Untimely response                       0
Submitted via_Fax                                                    0
Submitted via_Phone                                                  0
Submitted via_Postal mail                                            0
Submitted via_Referral                                               0
Submitted via_Web                                                    0
Product_Consumer Loan                                                0
Product_Credit card                                                  0
Produc

In [78]:
cd_train.head(4)

,Timely response?_Yes,Company response to consumer_Closed with explanation,Company response to consumer_Closed with monetary relief,Company response to consumer_Closed with non-monetary relief,Company response to consumer_Closed with relief,Company response to consumer_Closed without relief,Company response to consumer_Untimely response,Submitted via_Fax,Submitted via_Phone,Submitted via_Postal mail,...,State_PA,State_IL,State_VA,State_MD,State_OH,State_NC,State_MI,State_AZ,State_WA,Super
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,NaN
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,NaN
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [79]:
cd_train['Super']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
478416    NaN
478417    NaN
478418    NaN
478419    NaN
478420    NaN
Name: Super, Length: 478421, dtype: object

In [80]:
cd1_train,cd1_test=train_test_split(cd_train,test_size=0.2,random_state=2)

In [81]:
from sklearn.naive_bayes import MultinomialNB

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
lemma=WordNetLemmatizer()
stop=set(stopwords.words('english'))

In [83]:
def split_into_lemmas(message):
    words=word_tokenize(message)
    words_sans_stop=[]
    for word in words:
        if word in stop:continue
        words_sans_stop.append(word)
    return[lemma.lemmatize(word) for word in words_sans_stop]

In [84]:
tfidf=TfidfVectorizer(analyzer=split_into_lemmas,min_df=10,max_df=3000)

In [85]:
tfidf.fit(cd_train['Super'])

TfidfVectorizer(analyzer=<function split_into_lemmas at 0x2613BBB0>,
                binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=3000, max_features=None,
                min_df=10, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [86]:
train_data=tfidf.transform(cd_train['Super'])

In [87]:
train_data

<478421x6925 sparse matrix of type '<class 'numpy.float64'>'
	with 1006402 stored elements in Compressed Sparse Row format>

In [88]:
test_data=tfidf.transform(cd_test['Super'])

In [89]:
clf=MultinomialNB()

In [90]:
clf.fit(train_data,cd_train['Consumer disputed?'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [91]:
Final_data=tfidf.transform(cd_train['Super'])

In [92]:
p=clf.predict_proba(Final_data)

In [93]:
p.shape

(478421, 2)

In [94]:
p[:,0]

array([0.21201201, 0.21201201, 0.21201201, ..., 0.21201201, 0.21201201,
       0.21201201])

In [95]:
clf.classes_

array([0, 1])

In [96]:
cd_train['Tfidf']=np.where(p[:,0]>=0.5,0,1)

In [97]:
del cd_train['Super']

In [98]:
del cd_train['Sub-product']

In [99]:
Test_Final_data=tfidf.transform(cd_test['Super'])

In [100]:
Test_Final_data

<119606x6925 sparse matrix of type '<class 'numpy.float64'>'
	with 244927 stored elements in Compressed Sparse Row format>

In [101]:
r=clf.predict_proba(Test_Final_data)

In [102]:
r

array([[0.21201201, 0.78798799],
       [0.21201201, 0.78798799],
       [0.21201201, 0.78798799],
       ...,
       [0.21201201, 0.78798799],
       [0.21201201, 0.78798799],
       [0.21201201, 0.78798799]])

In [103]:
cd_test['Tfidf']=np.where(r[:,0]>=0.5,1,0)

In [104]:
del cd_test['Super']
del cd_test['Sub-product']

In [105]:
cd_train.dtypes

Timely response?_Yes                                              uint8
Company response to consumer_Closed with explanation              uint8
Company response to consumer_Closed with monetary relief          uint8
Company response to consumer_Closed with non-monetary relief      uint8
Company response to consumer_Closed with relief                   uint8
Company response to consumer_Closed without relief                uint8
Company response to consumer_Untimely response                    uint8
Submitted via_Fax                                                 uint8
Submitted via_Phone                                               uint8
Submitted via_Postal mail                                         uint8
Submitted via_Referral                                            uint8
Submitted via_Web                                                 uint8
Product_Consumer Loan                                             uint8
Product_Credit card                                             

In [106]:
cd1_train,cd1_test=train_test_split(cd_train,test_size=0.2,random_state=2)

In [107]:
x=cd1_train.drop(['Consumer disputed?','Complaint ID'],1)
y=cd1_train['Consumer disputed?']
k=cd1_test.drop(['Consumer disputed?','Complaint ID'],1)
m=cd1_test['Consumer disputed?']


In [108]:
x.shape

(382736, 42)

In [109]:
m.value_counts()

1    75566
0    20119
Name: Consumer disputed?, dtype: int64

In [82]:
y.shape

(382736,)

In [83]:
m.shape

(95685,)

In [110]:
from sklearn.linear_model import LogisticRegression

In [111]:
clf=LogisticRegression()

In [112]:
clf.fit(x,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [113]:
prediction=clf.predict(k)

In [114]:
from sklearn.metrics import roc_auc_score

In [115]:
prediction

array([1, 1, 1, ..., 1, 1, 1])

In [116]:
m

154142    1
206525    0
26341     1
398527    1
134537    1
         ..
437699    1
325670    1
293448    0
317292    1
73745     0
Name: Consumer disputed?, Length: 95685, dtype: int32

In [117]:
roc_auc_score(prediction,m)

0.7183267931337731

In [118]:
prediction=np.where(clf.predict(cd_test.drop(['Complaint ID'],1))==1,"Yes","No")

In [119]:
prediction

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype='<U3')

In [120]:
submission=pd.DataFrame(list(zip(cd_test['Complaint ID'],list(prediction))),
                       columns=['Complaint ID','Consumer disputed?'])

In [121]:
m.value_counts()

1    75566
0    20119
Name: Consumer disputed?, dtype: int64

In [122]:
submission['Consumer disputed?'].value_counts()

No     95737
Yes    23869
Name: Consumer disputed?, dtype: int64

In [123]:
submission.to_csv('submission_final.csv',index=False)

In [124]:
cd_train['Consumer disputed?'].value_counts()

1    376990
0    101431
Name: Consumer disputed?, dtype: int64